####  Getting Dataset from Kaggle Website

In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = 'your google drive directory'

In [ ]:
%cd  'your google drive directory'

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!unzip \*.zip && rm *.zip

**Preparing the data set**

The following cells resize each image to (150,150) grayscale image. 

In [ ]:
#TRAIN DATA

%tensorflow_version 1.x
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

import math
from tensorflow.python.framework import ops

def read_image(img):
    img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    img=np.array(img)
    img=cv2.resize(img,(150,150))
    img = img/255.0
    return img

train_data=[]
import os
for dirname, n, filenames in os.walk('./chest_xray/train/PNEUMONIA'):
    for file in filenames:
        if file == ".DS_Store":
            continue
        train_data.append([read_image(os.path.join("./chest_xray/train/PNEUMONIA",file)),1])
print("ONE")  
for dirname, n, filenames in os.walk('./chest_xray/train/NORMAL'):
    for file in filenames:
        if file == ".DS_Store":
            continue
        train_data.append([read_image(os.path.join('./chest_xrayy/train/NORMAL',file)),0])
             

In [ ]:
#TEST DATA
test_data=[]
for dirname, n, filenames in os.walk('./chest_xray/test/PNEUMONIA'):
    for file in filenames:
        test_data.append([read_image(os.path.join("./chest_xray/test/PNEUMONIA",file)),1])

for dirname, n, filenames in os.walk('./chest_xray/test/NORMAL'):
    for file in filenames:
        if file == ".DS_Store":
            continue
        test_data.append([read_image(os.path.join("./chest_xray/test/NORMAL",file)),0])
                

In [ ]:
#VAL DATA
val_data=[]
for dirname,n,filenames in os.walk("./chest_xray/PNEUMONIA"):
  for file in filenames:
        val_data.append([read_image(os.path.join("./chest_xray/PNEUMONIA",file)),1])

for dirname, n, filenames in os.walk('./chest_xray/val/NORMAL'):
    for file in filenames:
        if file == ".DS_Store":
            continue
        val_data.append([read_image(os.path.join("./chest_xray/val/NORMAL",file)),0])
                
  

In [ ]:
print("TOTAL TRAIN IMAGES: ",len(train_data))
print("TOTAL TEST IMAGES: ",len(test_data))
print("TOTAL VAL IMAGES: ",len(val_data))

In [ ]:
#split the dataset [imgs,label] into imgs and labels.
def generate_data(data):
    imgs= []
    labels=[]
    for i in (data):
        imgs.append(i[0])
        labels.append(i[1])
    return imgs,labels

train_images,train_labels = generate_data(train_data)
test_images,test_labels=generate_data(test_data)
val_images,val_labels=generate_data(val_data)

In [ ]:
train_images = np.array(train_images)
train_images = train_images.reshape(-1,150,150,1)
train_labels=np.array(train_labels)

test_images = np.array(test_images)
test_images = test_images.reshape(-1,150,150,1)
test_labels=np.array(test_labels)

val_images = np.array(val_images)
val_images = val_images.reshape(-1,150,150,1)
val_labels=np.array(val_labels)

In [ ]:
train_images = np.float32(train_images)
train_labels=np.float32(train_labels)

test_images = np.float32(test_images)
test_labels = np.float32(test_labels)

val_images = np.float32(val_images)
val_labels = np.float32(val_labels)


Convert the labels into one-hot encoded vector

In [ ]:
from tensorflow.python.keras.utils import to_categorical
train_labels=to_categorical(train_labels)
test_labels = to_categorical(test_labels)
val_labels = to_categorical(val_labels)

In [ ]:
print("TRAIN IMAGES SHAPE: ",train_images.shape)
print("TEST IMAGES SHAPE: ",test_images.shape)
print("VAL IMAGES SHAPE: ",val_images.shape)

In [ ]:
def mini_batches(train_images,train_labels,batch_size=128):
    m = len(train_images)
    permutation = list(np.random.permutation(m))
    shuffled_X = train_images[permutation,:,:,:]
    shuffled_Y = train_labels[permutation,:]
    num_batches = int(np.floor(m/batch_size))
    batches=[]
    for i in range(num_batches):
        batch_x = shuffled_X[i * batch_size : i * batch_size + batch_size,:,:,:]
        batch_y = shuffled_Y[i * batch_size :i*batch_size + batch_size,:]
        batches.append([batch_x,batch_y])
        
    return batches

## **Plotting random Images from train Dataset**

In [ ]:
r=0
from matplotlib import style
style.use('dark_background')
fig,ax = plt.subplots(5,5,figsize=(20,20))
for i in range(ax.shape[0]):
    for u in range(ax.shape[1]):
        r=np.random.randint(1,500)
        ax[i,u].imshow(np.squeeze(train_images[r]),cmap='gray')
        if int(np.argmax(train_labels[r])) ==0:
            title = "NORMAL"
        elif int(np.argmax(rain_labels[r])) ==1:
            title="PNEUMONIA"
        ax[i,u].set_title(title) 

## **Building the Model**

In [ ]:
def conv2d(X,weights,bias,padding="SAME",strides = 1):
    z = tf.nn.conv2d(X,weights,strides=[1,strides,strides,1],padding="SAME")
    z = tf.nn.bias_add(z,bias)
    activation = tf.nn.relu(z)
    return activation

In [ ]:
def weights_initializer(nclasses):
    weights = {"W1" : tf.get_variable("W1",shape =(3,3,1,32),
                                     initializer = tf.contrib.layers.xavier_initializer()),
              "W2":tf.get_variable("W2",shape = (3,3,32,64),
                                  initializer = tf.contrib.layers.xavier_initializer()),
              "W3":tf.get_variable("W3",shape = (3,3,64,128),
                                  initializer = tf.contrib.layers.xavier_initializer()),
               #"W4":tf.get_variable("W4",shape=(3,3,128,128),
                                   #initializer = tf.contrib.layers.xavier_initializer()),
              "D1":tf.get_variable("D1",shape = (19*19*128,128),
                                  initializer = tf.contrib.layers.xavier_initializer()),
              "D2":tf.get_variable("D2",shape = (128,512),
                                  initializer = tf.contrib.layers.xavier_initializer()),
              "D3":tf.get_variable("D3",shape = (512,nclasses),
                                  initializer = tf.contrib.layers.xavier_initializer())}
    bias = {"B1" :tf.get_variable("B1",shape=(32),initializer = tf.zeros_initializer()),
           "B2":tf.get_variable("B2",shape = (64),initializer = tf.zeros_initializer()),
           "B3":tf.get_variable("B3",shape = (128),initializer = tf.zeros_initializer()),
           #"B4":tf.get_variable("B4",shape = (128),initializer = tf.zeros_initializer()),
           'D1': tf.get_variable('B5', shape=(128), initializer=tf.zeros_initializer()),
           'D2': tf.get_variable('B6', shape=(512), initializer=tf.zeros_initializer()),
           'D3': tf.get_variable('B7', shape=(nclasses), initializer=tf.zeros_initializer()),

}
    return weights,bias


In [ ]:
def conv_net(inp_image,weights,bias):
    with tf.name_scope("Network"):
        #Conv_layer
        conv1 = conv2d(inp_image,weights["W1"],bias["B1"])
        pool1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1],strides = [1,2,2,1],padding="SAME")
        conv2 = conv2d(pool1,weights["W2"],bias["B2"])
        pool2 = tf.nn.max_pool(conv2,ksize = [1,2,2,1],strides = [1,2,2,1],padding = "SAME")
        conv3 = conv2d(pool2,weights["W3"],bias["B3"])
        pool3 = tf.nn.max_pool(conv3,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
        #conv4 = conv2d(pool3,weights["W4"],bias["B4"])
       # pool4 = tf.nn.max_pool(conv4,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
        
        #Flatten 
        flatten = tf.contrib.layers.flatten(pool3)

        #Dense
        Z1 = tf.add(tf.matmul(flatten,weights["D1"]),bias["D1"])
        A1 = tf.nn.relu(Z1)
        A1 = tf.nn.dropout(A1, keep_prob=1-0.25)
        
        Z2 = tf.add(tf.matmul(A1,weights["D2"]),bias["D2"])
        A2 = tf.nn.relu(Z2)
        A2 = tf.nn.dropout(A2,keep_prob=1-0.25)
        
        Z3 = tf.add(tf.matmul(A2,weights["D3"]),bias["D3"])

    return Z3

In [ ]:
def placeholders(img_size,nclasses,nchannels):
    X_train = tf.placeholder(tf.float32,shape=(None,img_size,img_size,nchannels))
    y_train = tf.placeholder(tf.float32,shape=(None,nclasses))
    return X_train,y_train

In [ ]:
def logits_and_loss(X_train,y_train,weights,bias):
    normal = 0
    pneumonia= 0
    for i in train_labels:
        if np.argmax(i) ==0:
            normal+=1
        elif np.argmax(i)==1:
            pneumonia+=1
    weights_for_normal = (1 / normal)*(len(train_images))/2.0
    weights_for_pneumonia = (1 / pneumonia)*(len(train_images))/2.0
    class_weight = tf.constant([weights_for_normal, weights_for_pneumonia])
    logits = conv_net(X_train,weights,bias)
    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits = logits,
                                                                    labels = y_train,pos_weight = class_weight))
    return loss,logits


In [ ]:
ops.reset_default_graph() #Reset graph
def train(train_images,train_labels,test_images,test_labels,
         learning_rate = 0.001,epochs=5,batch_size=64):
    nclasses=2
    img_size = 150
    nchannels = train_images.shape[-1]
    #initialize weights 
    weights,bias= weights_initializer(nclasses)
    #Placeholder
    X_train,y_train = placeholders(img_size,nclasses,nchannels)
    loss,_ = logits_and_loss(X_train,y_train,weights,bias)
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
    tf.summary.scalar("Training_Loss_:",loss)
    A2 =  conv_net(X_train,weights,bias)
    test_predictions = tf.nn.softmax(A2)
    correct_prediction = tf.equal(tf.argmax(y_train, 1), tf.argmax(test_predictions, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Validation_accuracy_:",accuracy)

    merge_summary=tf.summary.merge_all()
    init=tf.global_variables_initializer()

    with tf.Session() as sess:  
        sess.run(init)
        summary_writer = tf.summary.FileWriter('graphs/',graph = tf.get_default_graph())
        for i in range(epochs):
            minibatch_cost = 0
            minibatches=random_mini_batches(train_images, train_labels,mini_batch_size = 128)
            for minibatch in minibatches:
                batch_images,batch_labels =minibatch 
                feed_dict = {X_train:batch_images,y_train:batch_labels}
                _,temp_cost,summary = sess.run([optimizer,loss,merge_summary],feed_dict = feed_dict)
                minibatch_cost += temp_cost / len(minibatches)
                summary_writer.add_summary(summary,i)
            val_acc = sess.run(accuracy,feed_dict={X_train: val_images, y_train: val_labels})
            
            print('\nEpoch {} \n Loss: {} Validation Accuracy: {}'.format(i+1, minibatch_cost,val_acc))       

In [ ]:
train(train_images,train_labels,test_images,test_labels,
         learning_rate = 1e-1,epochs=25)

**Load Tensorboard**

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir 'logs files path'